In [5]:
import pandas as pd

# --- Handcrafted OHLCV Data ---
# We create a story for 'TICKA' and 'TICKB' over several days.
data = {
    'Ticker': ['TICKA', 'TICKA', 'TICKA', 'TICKA', 'TICKA', 
               'TICKB', 'TICKB', 'TICKB', 'TICKB', 'TICKB'],
    'Date': pd.to_datetime(['2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06',
                           '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06']),
    'Adj High':  [100, 101, 102, 98,  95,   50,  51,  58,  59,  55], # Day 2: Entry price for TICKA is 101.
    'Adj Low':   [98,  99,  100, 94,  93,   49,  50,  56,  57,  52], # Day 5: Exit price for TICKA is 93.
    'Adj Close': [99,  100, 101, 95,  94,   50,  50,  57,  58,  53]  # Day 4: Close of 95 triggers stop-loss for TICKA.
}
test_df_ohlcv = pd.DataFrame(data).set_index(['Ticker', 'Date'])

# 1. make a clean copy of TICKB
tickc = (
    test_df_ohlcv.loc['TICKB']          # pull TICKB slice (drops 1st index level)
    .copy()
    .assign(Ticker='TICKC')             # re-label
    .set_index('Ticker', append=True)   # put Ticker back
    .swaplevel()                        # -> (Ticker, Date)
)

# 2. drop any stale TICKC rows
test_df_ohlcv = test_df_ohlcv.drop('TICKC', level='Ticker', errors='ignore')

# 3. append the single, clean TICKC block
test_df_ohlcv = pd.concat([test_df_ohlcv, tickc]).sort_index()

# # 4. now do your one-cell edit
test_df_ohlcv.loc[('TICKC', ['2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06']), 
                   ['Adj High', 'Adj Low', 'Adj Close']] = [50, 48, 49]

print("--- Test Data and Config ---")
print("\nOHLCV DataFrame:")
print(test_df_ohlcv)



# print("--- Test Data and Config ---")
# print("Config:", test_config)
# print("\nOHLCV DataFrame:")
# print(test_df_ohlcv)

--- Test Data and Config ---

OHLCV DataFrame:
                   Adj High  Adj Low  Adj Close
Ticker Date                                    
TICKA  2023-01-02       100       98         99
       2023-01-03       101       99        100
       2023-01-04       102      100        101
       2023-01-05        98       94         95
       2023-01-06        95       93         94
TICKB  2023-01-02        50       49         50
       2023-01-03        51       50         50
       2023-01-04        58       56         57
       2023-01-05        59       57         58
       2023-01-06        55       52         53
TICKC  2023-01-02        50       48         49
       2023-01-03        50       48         49
       2023-01-04        50       48         49
       2023-01-05        50       48         49
       2023-01-06        50       48         49


In [6]:
df = test_df_ohlcv.reset_index()   

In [22]:
list(pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d'))

['2023-01-02',
 '2023-01-03',
 '2023-01-04',
 '2023-01-05',
 '2023-01-06',
 '2023-01-02',
 '2023-01-03',
 '2023-01-04',
 '2023-01-05',
 '2023-01-06',
 '2023-01-02',
 '2023-01-03',
 '2023-01-04',
 '2023-01-05',
 '2023-01-06']

In [25]:
_list =   df['Adj Close'].tolist()
print(_list)

[99, 100, 101, 95, 94, 50, 50, 57, 58, 53, 49, 49, 49, 49, 49]


In [27]:
import pandas as pd

# --- extract the current MultiIndex DataFrame back to a plain dict ---
df = test_df_ohlcv.reset_index()          # flatten index -> columns

data = {
    'Ticker': ['TICKA', 'TICKA', 'TICKA', 'TICKA', 'TICKA', 
               'TICKB', 'TICKB', 'TICKB', 'TICKB', 'TICKB', 
               'TICKC', 'TICKC', 'TICKC', 'TICKC', 'TICKC'],
    'Date': ['2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06',
             '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06',
             '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06'],
    'Adj High': [100, 101, 102, 98, 95, 50, 51, 58, 59, 55, 50, 50, 50, 50, 50],
    'Adj Low': [98, 99, 100, 94, 93, 49, 50, 56, 57, 52, 48, 48, 48, 48, 48],
    'Adj Close': [99, 100, 101, 95, 94, 50, 50, 57, 58, 53, 49, 49, 49, 49, 49],
}

# (optional) rebuild the same MultiIndex frame
test_df_ohlcv = pd.DataFrame(data).set_index(['Ticker', 'Date'])

print("--- Test Data and Config ---")
print("\nOHLCV DataFrame:")
print(test_df_ohlcv)

--- Test Data and Config ---

OHLCV DataFrame:
                   Adj High  Adj Low  Adj Close
Ticker Date                                    
TICKA  2023-01-02       100       98         99
       2023-01-03       101       99        100
       2023-01-04       102      100        101
       2023-01-05        98       94         95
       2023-01-06        95       93         94
TICKB  2023-01-02        50       49         50
       2023-01-03        51       50         50
       2023-01-04        58       56         57
       2023-01-05        59       57         58
       2023-01-06        55       52         53
TICKC  2023-01-02        50       48         49
       2023-01-03        50       48         49
       2023-01-04        50       48         49
       2023-01-05        50       48         49
       2023-01-06        50       48         49
